# Camelyon_16_extract_patch

##### Import modules

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset
from torch.nn import functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import torch.backends.cudnn as cudnn
from torch.nn.utils.rnn import pack_sequence
from PIL import Image
from sklearn.manifold import TSNE
import cv2 
import openslide
from openslide import OpenSlide
from openslide import open_slide
from skimage.io import imread, imshow
from skimage.color import rgb2gray, rgb2hsv
from skimage.morphology import area_opening
from skimage.exposure import histogram
from skimage.filters import threshold_otsu
from skimage.color import rgb2hsv
import numpy as np
import tqdm
import os
import time
import copy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

/pasteur/appa/homes/cbacquie/miniconda3/envs/camelyon/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


i=1
image_path=f"zeus/Data/Camelyon/tumor/images/{i:03}/tumor_{i:03}.tif"
image_path2=f"zeus/Data/Camelyon/tumor/images/{i:03}/tumor_{i:03}.png"
img=OpenSlide(image_path)
thumbnail=img.get_thumbnail((1000,1000))
thumbnail=np.array(thumbnail)
thumbnail= thumbnail.astype(np.uint8)
Image.fromarray(thumbnail).save(image_path2)
a=torch.load(f"zeus/Data/Camelyon/tumor/features/{i+1:03}/feat_tumor_{i+1:03}.npy")
print(a)

#### Create_mask_Segmentation

In [2]:
def get_mask1(thumbnail,path_mask1):
    """conversion of the image in hsv format, mask on the saturation channel with an otsu filter
    thumbnail: thumbnail created with get.thumbnail function
    path_mask1:path in which we want to save our mask
    return:save the mask as jpg in the given folder and return the mask """
    hsv = cv2.cvtColor(thumbnail, cv2.COLOR_BGR2HSV)
    plt.imshow(hsv[:,:,1])
    ret,thresh1 = cv2.threshold(hsv[:,:,1],0,255,cv2.THRESH_OTSU)
    images = [hsv, thresh1]
    titles=["Image","Treshold"]
    plt.figure(figsize=(10,10))
    for i in range(2):
        plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
        plt.title(titles[i])
        plt.xticks([]),plt.yticks([])
    plt.figure(figsize=(10,10))
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    fg_mask1=closing
    #Image.fromarray(closing).save(path_mask1)
    return fg_mask1

In [3]:
def get_final_mask(thumbnail,fg_mask1,path_final_mask):
    """superposition of the first mask and our image in order to remove the black elements then otsu filter
    thumbnail: thumbnail created with get.thumbnail function,the same as the one used for the first mask
    path_final_mask:path in which we want to save our mask
    return:save the mask as jpg in the given folder and return the mask """
    thumbnail2 = thumbnail*np.expand_dims(fg_mask1.astype(np.float32)/255,axis=2).astype(np.uint8)
    gray = cv2.cvtColor(thumbnail2, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0).astype(np.uint8)
    # Image.fromarray(blur).save("Data/001/patch/blur.jpg")
    ret,thresh1 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    fg_mask=closing
    #Image.fromarray(fg_mask).save(path_final_mask)
    fg_mask=closing/255
    return fg_mask

#### Extract patches

In [4]:
def _edge_case(image_shape, y, x, patch_size, step_size):
    """Keep the tile size at TILE_HEIGHT, TILE_WIDTH """
    vertical_limit = image_shape[0]-patch_size
    horizontal_limit = image_shape[1]-patch_size
    new_y = max(0, min(y, vertical_limit))
    new_x = max(0, min(x, horizontal_limit))
    return new_y,new_x


In [5]:
def basic_patchify_mask(patch_size, overlap, mask, on_mask=0.01):
    """A simple patches division in grid"""
    patch_dict={}
    step_size = int(patch_size-overlap)                                    
    index = 0
    for y in range(0,mask.shape[0], step_size):                                      
        for x in range(0,mask.shape[1], step_size):
            # Moves back tile if necessary so they are all of the same size                           
            y, x = _edge_case(mask.shape, y, x, patch_size, step_size)
            # Fraction of masked pixels
            if np.mean(mask[y:y+patch_size, x:x+patch_size]) >= on_mask:
                corner = [x, y] 
                patch = {"corner": corner}
                patch_dict[index] = patch
                index += 1
    return patch_dict

In [6]:
def patchify_mask(patch_size, overlap, mask, scale=1., on_mask=0.01):
    """
    Adapt patches parameters to the mask scale and patchify.
    For best performance, scale, patch_size and overlap should be powers of 2
    Scale: original_image_size/mask_size
    """
    scaled_patch_size = int(patch_size//scale)
    scaled_overlap = int(overlap//scale)
    patch_dict = basic_patchify_mask(scaled_patch_size, scaled_overlap, mask, on_mask=on_mask)
    # Rescale
    rescaled_patch_dict = {}
    for index, patch in patch_dict.items():
        corner = patch.get("corner")
        rescaled_corner = [int(corner[0]*scale), int(corner[1]*scale)]
        rescaled_patch_dict[index] = {"corner": rescaled_corner, "size": patch_size}
    return rescaled_patch_dict


In [7]:
def pixels_extraction_numpy(image, patch):
    """Get the pixel information for a patch"""
    corner = patch.get("corner")
    patch_size = patch.get("size")
    # Numpy uses [y, x, c] axis notation, corner is [x, y] 
    corner.reverse()
    pixels = image[
            corner[0]:corner[0]+patch_size,
            corner[1]:corner[1]+patch_size,
            ...
            ]
    return pixels

def pixels_extraction_openslide(image, patch, level=0):
    """Get the pixel information for every patch"""
    corner = patch.get("corner")
    patch_size = patch.get("size")
    pixels = image.read_region(location=corner, level=level, size=(patch_size, patch_size))
    return pixels


In [8]:
def visualize_patches(image, patch_dict,display_path, line_width=20, scale=1.):
    from random import randint
    import cv2
    display_image = np.asarray(image)
    for index, patch in patch_dict.items():
        color = [randint(0,255) for _ in range(3)]
        scaled_patch_corner = [int(coord//scale) for coord in patch["corner"]]
        scaled_patch_size = int(patch["size"]//scale)
        bottom_right_corner = [coord+scaled_patch_size for coord in scaled_patch_corner]
        # Draw the rescaled patch rectangles
        cv2.rectangle(display_image, scaled_patch_corner, bottom_right_corner, color, line_width)
        #Image.fromarray(display_image).save(display_path)
    return display_image

In [9]:
def save_patches(image,patch_dict, directory,basename,level):
    """Save all images in the patch dictionary
    save all images in the directory in .png"""
    L=[]
    i=0
    for index, patch in tqdm.tqdm(patch_dict.items()):
        L.append([i,patch_dict[index]])
        i+=1
        pixels = pixels_extraction_openslide(image, patch, level=level)
        pixels = pixels.convert('RGB')
        pixels=pixels.resize((224,224),resample=Image.BILINEAR)
        pixels.save((os.path.join(directory, basename+str(index)+'.png')))
    return L
        
        

In [10]:
def see_patch(directory, basename):
    """show 225 first patches"""
    fig=plt.figure(figsize=(30,30)) 
    rows = 15
    columns = 15
    for i in range(1,226):
        image = mpimg.imread(os.path.join(directory, basename+str(i)+'.png'))
        # print(image.min(),image.max())
        a=fig.add_subplot(rows, columns, i) 
        a.imshow(image) 
        plt.axis('off') 
        plt.title(str(i))

In [ ]:
def main(i):
    ####Parameters####
    patch_size=500
    tb_size=1000
    overlap=0
    level=0
    ####images####
    image_path=f"zeus/Data/Camelyon/normal/images/{i:03}/normal_{i:03}.tif"
    img=OpenSlide(image_path)
    directory_patch=f"zeus/Data/Camelyon/normal/images/{i:03}/patch"
    path_mask1=f"zeus/Data/Camelyon/normal/images/{i:03}/mask/mask1.jpg"
    path_final_mask=f"zeus/Data/Camelyon/normal/images/{i:03}/mask/final_mask.jpg"
    basename=f"normal_{i:03}_patch"
    thumbnail=img.get_thumbnail((1000,1000))
    thumbnail.show()
    display_path=f"zeus/Data/Camelyon/normal/images/{i:03}/patch/display_patches.jpg"
    thumbnail=np.array(thumbnail)
    #print(thumbnail.min(),thumbnail.max())
    thumbnail= thumbnail.astype(np.uint8)
    ####Parameters####
    im_width, im_height = img.dimensions
    process_scale = max(im_width,im_height)/1000
    scale = img.level_downsamples[level]
    ####Get mask1####
    fg_mask1=get_mask1(thumbnail,path_mask1)
    print("ok_mask1")
    ####Get final mask####
    fg_mask=get_final_mask(thumbnail,fg_mask1,path_final_mask)
    print("ok_final_mask")
    ####Get patches####
    patch_dict = patchify_mask(int(patch_size*scale),
        overlap=overlap, mask=fg_mask, scale=process_scale)
    for patch in patch_dict.values():
        patch["size"] = int(patch["size"]//scale)
    print("ok_patch_dict")
    ####Visualize patch on the image####
    visualize_patches(thumbnail, patch_dict,display_path, scale=process_scale, line_width=5)
    #print("ok_display")
    ####Extract pixels and save####
    L=save_patches(img, patch_dict,directory_patch,basename,level=level)
    np.save(f"zeus/Data/Camelyon/normal/images/{i:03}/patch_dict.npy",L)
    print("ok_save")
    ####Visualize patch####
    print(see_patch(directory_patch,basename))
if __name__=="__main__":
    main(34)

ok_mask1
ok_final_mask
ok_patch_dict


  6%|████▌                                                                          | 649/11199 [00:28<08:51, 19.85it/s]

##### Create Dataset

In [11]:
data_transforms = {"tumor" : transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        "normal":  transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        "test": transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }
data_dir="zeus/Data/Camelyon/"
image_datasets={}
dataset_sizes = {}
for x in tqdm.tqdm([f"tumor/images/{i:03}/" for i in range(1,61)]):
    image_datasets[x]=datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms["tumor"])
    dataset_sizes[x] = len(image_datasets[x])

    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:32<00:00,  1.61s/it]


In [12]:
for x in tqdm.tqdm([f"normal/images/{i:03}/" for i in range(1,61)]):
    image_datasets[x]=datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms["normal"])
    dataset_sizes[x] = len(image_datasets[x])
#for x in [f"test/{i:03}/patch" for i in range(1,21)]:
 #   image_datasets[x]=datasets.ImageFolder(os.path.join(data_dir, x),
  #                                        data_transforms["test"])
   # dataset_sizes[x] = len(image_datasets[x])

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.18it/s]


In [13]:
for x in tqdm.tqdm([f"tumor/images/{i:03}/" for i in range(1,61)]):
    for i in range(len(image_datasets[x].targets)):
        image_datasets[x].targets[i]=0 #tumor=0
for x in tqdm.tqdm([f"normal/images/{i:03}/" for i in range(1,61)]):
    for i in range(len(image_datasets[x].targets)):
        image_datasets[x].targets[i]=1 #normal=1

100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 3644.37it/s]


#### Create Dataloaders

In [14]:
dataloaders ={}
for x in tqdm.tqdm([f"tumor/images/{i:03}/" for i in range(1,61)]):
    dataloaders[x]=torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=False, num_workers=0)
for x in tqdm.tqdm([f"normal/images/{i:03}/" for i in range(1,61)]):
    dataloaders[x]=torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=False, num_workers=0)
#for x in [f"test/{i:03}/patch" for i in range(1,21)]:
 #   dataloaders[x]=torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
  #                                           shuffle=False, num_workers=4)
    

100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 33487.46it/s]


#### Extract Features

In [15]:
model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Identity(num_ftrs, 2)
model_conv = model_conv.to(device)

/pasteur/appa/homes/cbacquie/miniconda3/envs/camelyon/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/pasteur/appa/homes/cbacquie/miniconda3/envs/camelyon/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
def visualize_model(model,dataloader):
    L_features=[]
    model.eval()
    with torch.no_grad():
        for i, (inputs,labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            L_features.append(outputs)
    return L_features

In [17]:
L_features_normal=[]
L_features_tumor=[]
for x in tqdm.tqdm([f"normal/images/{i:03}/" for i in range(1,61)]):
    L_features_normal.append(visualize_model(model_conv,dataloaders[x]))
for x in tqdm.tqdm([f"tumor/images/{i:03}/" for i in range(1,61)]):
    L_features_tumor.append(visualize_model(model_conv,dataloaders[x]))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [59:08<00:00, 177.44s/it]


In [18]:
for i in tqdm.tqdm(range(0,60)):
    features_tumor=torch.vstack(L_features_tumor[i])
    features_tumor.shape
    torch.save(features_tumor,f"zeus/Data/Camelyon/tumor/features/{i+1:03}/feat_tumor_{i+1:03}.npy")

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  3.07it/s]


In [19]:
for i in tqdm.tqdm(range(0,60)):
    features_normal=torch.vstack(L_features_normal[i])
    features_normal.shape
    torch.save(features_normal,f"zeus/Data/Camelyon/normal/features/{i+1:03}/feat_normal_{i+1:03}.npy")

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 10.72it/s]


In [20]:
#verification qu'on a bien des tensors de taille nx2048 
L=[]
for i in tqdm.tqdm(range(0,60)):
    a=torch.load(f"zeus/Data/Camelyon/normal/features/{i+1:03}/feat_normal_{i+1:03}.npy")
    L.append(a.shape)
print(L)
#ok

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.46it/s]

[torch.Size([1852, 2048]), torch.Size([1188, 2048]), torch.Size([4743, 2048]), torch.Size([310, 2048]), torch.Size([1335, 2048]), torch.Size([1226, 2048]), torch.Size([3118, 2048]), torch.Size([597, 2048]), torch.Size([7349, 2048]), torch.Size([1987, 2048]), torch.Size([11148, 2048]), torch.Size([3339, 2048]), torch.Size([609, 2048]), torch.Size([2069, 2048]), torch.Size([4802, 2048]), torch.Size([252, 2048]), torch.Size([407, 2048]), torch.Size([2965, 2048]), torch.Size([241, 2048]), torch.Size([558, 2048])]


In [ ]:
#verification qu'on a bien des tensors de taille nx2048 
L=[]
for i in tqdm.tqdm(range(0,60)):
    a=torch.load(f"zeus/Data/Camelyon/tumor/features/{i+1:03}/feat_tumor_{i+1:03}.npy")
    L.append(a.shape) 
print(L)
#ok

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.53it/s]

[torch.Size([6358, 2048]), torch.Size([3952, 2048]), torch.Size([5795, 2048]), torch.Size([11532, 2048]), torch.Size([3570, 2048]), torch.Size([15334, 2048]), torch.Size([11017, 2048]), torch.Size([11729, 2048]), torch.Size([10463, 2048]), torch.Size([8533, 2048]), torch.Size([4609, 2048]), torch.Size([6346, 2048]), torch.Size([5078, 2048]), torch.Size([9677, 2048]), torch.Size([27134, 2048]), torch.Size([4631, 2048]), torch.Size([8407, 2048]), torch.Size([8740, 2048]), torch.Size([2275, 2048]), torch.Size([6218, 2048])]


#### Create Dataset Features

In [2]:
#pandas dataset filename+y
datadict={}
Lfilename=[]
Ly=[]
for x in [f"zeus/Data/Camelyon/normal/features/{i+1:03}/feat_normal_{i+1:03}.npy" for i in range(0,60)]:
    Lfilename.append(x)
    Ly.append(1) #normal=1
    
for x in [f"zeus/Data/Camelyon/tumor/features/{i+1:03}/feat_tumor_{i+1:03}.npy" for i in range(0,60)]:
    Lfilename.append(x)
    Ly.append(0) #tumor=0
datadict['filename']=Lfilename
datadict['y']=Ly
dataset= pd.DataFrame(data=datadict)
#dataset["filename"][3]
#dataset[dataset["filename"]==dataset["filename"][3]]["y"][3]

1

In [6]:
class CustomImageDataset(Dataset):  #générateur
    def __init__(self, df_filename):
        self.df_filename = df_filename
    def __len__(self):
        return len(self.df_filename)
    def __getitem__(self, idx):
        filename_label = self.df_filename.loc[idx]
        filename=filename_label["filename"]
        label=filename_label["y"]
        matrix = torch.load(filename,map_location="cpu") #load features
        #print(matrix.shape,flush=True)
        #indices=np.random.choice(np.arange(matrix.shape[0]),5000, replace=False)#prendre 5000 patchs dans la matrice  
        #matrix=matrix[indices]
        return matrix, label
dataset_features={'train':CustomImageDataset(dataset)}
dataset_sizes=len(dataset_features['train'])
#CustomImageDataset(dataset)[4][1]

1

#### Create Dataloaders Features

In [4]:
#dataloaders
def collate_fn(batch):    
    data = [item[0] for item in batch]
    data = pack_sequence(data, enforce_sorted=False)[0]
    targets = torch.Tensor([item[1] for item in batch])
    return [data, targets]

dataloaders_features={'train':torch.utils.data.DataLoader(dataset_features['train'], 
                                batch_size=1, num_workers=1, shuffle=True)}
next(iter(dataloaders_features['train']))[0].shape


torch.Size([1, 8740, 2048])

#### Convolutionnal 1-1 Layer+min,max Layer+MLP

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv2d=nn.Conv2d(in_channels=2048, out_channels=1, kernel_size=1)
        mlp1 = nn.Linear(10, 200)
        mlp2 = nn.Linear(200, 100)
        mlp3 = nn.Linear(100, 1)
        self.mlp = nn.Sequential(mlp1,nn.Sigmoid(),nn.Dropout(0.2),
                                 mlp2,nn.Sigmoid(),nn.Dropout(0.2),
                                 mlp3)
    def forward(self, x):
        x=self.conv2d(x)
        #print(x.shape)
        x=x.reshape(1,-1)
        #
        min_x,indicemin=(torch.topk(x, 5, dim=1, largest=False,sorted=True))
        max_x,indicemax=(torch.topk(x, 5, dim=1, largest=True,sorted=True))
        output=torch.hstack((min_x,max_x))
        #print(output.shape)
        y_pred = self.mlp(output)
        return y_pred
                              

#### Training

In [12]:
model = Network().to(device)
criterion = nn.BCEWithLogitsLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    inputs=inputs.reshape(1,2048,-1, 1)
                    #print(inputs.shape)
                    outputs = model(inputs)
                    preds=torch.sigmoid(outputs)>0.5
                    loss = criterion(outputs, labels.unsqueeze(0).float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes
            epoch_acc = running_corrects.double() / dataset_sizes

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'train' and epoch_acc > best_acc: #remettre valid apres
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
print(train_model(model, criterion, optimizer, exp_lr_scheduler, dataloaders_features, num_epochs=200))


Epoch 0/199
----------
train Loss: 0.6988 Acc: 0.4750

Epoch 1/199
----------
train Loss: 0.7138 Acc: 0.4750

Epoch 2/199
----------
train Loss: 0.7181 Acc: 0.4250

Epoch 3/199
----------
train Loss: 0.6987 Acc: 0.5750

Epoch 4/199
----------
train Loss: 0.6963 Acc: 0.5500

Epoch 5/199
----------
train Loss: 0.7210 Acc: 0.4500

Epoch 6/199
----------
train Loss: 0.6894 Acc: 0.5250

Epoch 7/199
----------
train Loss: 0.7235 Acc: 0.3750

Epoch 8/199
----------
train Loss: 0.6956 Acc: 0.6250

Epoch 9/199
----------
train Loss: 0.6987 Acc: 0.5250

Epoch 10/199
----------
train Loss: 0.7034 Acc: 0.4750

Epoch 11/199
----------
train Loss: 0.6807 Acc: 0.6750

Epoch 12/199
----------
train Loss: 0.6961 Acc: 0.4750

Epoch 13/199
----------
train Loss: 0.7023 Acc: 0.4750

Epoch 14/199
----------
train Loss: 0.6975 Acc: 0.4250

Epoch 15/199
----------
train Loss: 0.7037 Acc: 0.5500

Epoch 16/199
----------
train Loss: 0.6968 Acc: 0.5500

Epoch 17/199
----------
train Loss: 0.6962 Acc: 0.5250

Ep

In [11]:
device="cpu"
model = Network().to(device)
for inputs, labels in dataloaders_features['train']:
                inputs = inputs.to(device)
                inputs=inputs.reshape(1,2048,-1, 1)
                labels = labels.to(device)
                a=model.conv2d(inputs)
                torch.histc(a)


In [ ]:
local_labels = []
local_scores = []
coords = []

for img, g, labels, xy in validation_dataloader:
    scores = model.embedding(img).view(-1).detach().cpu().numpy()
    local_scores.append(scores)
    local_labels.append(labels.view(-1).numpy())
    coords.append(xy)
    
    
    if g == 1:
#         plt.figure(figsize=(20,10))
#         plt.hist(scores, bins=100, density=True)
#         plt.yscale("log")
#         plt.show()
        plotMap(labels.view(-1).numpy(), scores<-0., xy[0].cpu().numpy())

# local_labels, local_scores = np.array(local_labels), np.array(local_scores)

In [ ]:
echnatillonnage uniforme ne pas prendre les images ou il y a peu